### 對於地址相同且經緯度距離在 20 公尺以內的記錄，僅保留一筆

In [2]:
import numpy as np
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    計算兩點經緯度之間的距離（單位：公尺）
    """
    R = 6371000  # 地球半徑 (公尺)
    
    # 將角度轉為弧度
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    
    # Haversine 公式
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c

def filter_close_points(group, threshold=20):
    """
    接收一個 group (同一個 address 的資料)，回傳需要保留的 index
    """
    
    # 解析經緯度 "119.884910,31.235518" -> lon, lat
    # 注意：split後轉float
    coords = group['location'].str.split(',', expand=True).astype(float)
    lons = coords[0].values  # 經度
    lats = coords[1].values  # 緯度
    indices = group.index.values
    
    keep_indices = []
    dropped_mask = np.zeros(len(group), dtype=bool) # 標記是否已被剔除
    
    for i in range(len(group)):
        if dropped_mask[i]:
            continue
            
        # 保留當前這一筆 (作為基準點)
        keep_indices.append(indices[i])
        
        # 計算當前點 (i) 與所有點的距離
        dists = haversine_distance(lats[i], lons[i], lats, lons)
        
        # 找出距離小於門檻值且尚未被處理的點 (包含自己，但沒關係因為loop會往下跑)
        # 標記這些點為 "已剔除" (因為它們離基準點太近了)
        close_hits = np.where(dists <= threshold)[0]
        dropped_mask[close_hits] = True
        
    return keep_indices


### 商务住宅

In [3]:


# 1. 讀取 CSV (請確認編碼，中文常是 utf-8-sig 或 gb18030)
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_商务住宅.csv'  # 你的檔案名稱
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 1305
篩選後筆數: 1299
                                        id pname   pcode cityname  citycode  \
0     3afcb146-d32d-3622-8838-5e6025dfcffa   江苏省  320000      无锡市       510   
1     ef1ac6e5-99f4-3cab-8b71-1860d3b4825b   江苏省  320000      无锡市       510   
2     59474927-1d35-3859-bd15-ec527d7f06a3   江苏省  320000      无锡市       510   
3     6e2c351f-8dfe-302b-8b68-e5932c16ee8a   江苏省  320000      无锡市       510   
4     c295a58b-d914-3463-8c76-207003da92de   江苏省  320000      无锡市       510   
...                                    ...   ...     ...      ...       ...   
1300  44c287df-eb4d-3ccf-b6ef-54c39acd3aad   江苏省  320000      无锡市       510   
1301  383fc298-d3dc-3017-84ad-84aa0ba166c7   江苏省  320000      无锡市       510   
1302  f348ff07-7bb4-3140-bc91-b342637f7436   江苏省  320000      无锡市       510   
1303  ac88f801-98fc-3a45-8167-110b158cd3a0   江苏省  320000      无锡市       510   
1304  907fb253-090b-3952-9583-e3b171f52850   江苏省  320000      无锡市       510   

     adname  adcode        n

In [4]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆資料到 {output_file}")

成功追加 1299 筆資料到 location_sites.csv


### 交叉路口

In [5]:
# 1. 讀取 CSV (請確認編碼)
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_地名地址信息.csv' 
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')

print(f"原始資料筆數: {len(df)}")

# ==========================================
# [新增邏輯]：只保留 "name" 中有 "交叉口" 的資料
# ==========================================
# na=False 參數確保如果 name 欄位是空值(NaN)，不會報錯，而是直接視為不包含
df = df[df['name'].str.contains('交叉口', na=False)]

print(f"包含'交叉口'的資料筆數: {len(df)}")


# 2. 對 address 進行 groupby
# 3. 對每個 group 應用篩選邏輯
indices_to_keep = []

# 因為前面已經篩選過 df，這裡的 group 就全都是包含 "交叉口" 的資料了
for name, group in df.groupby('address'):
    # 注意：請確保 filter_close_points 函式存在並能回傳 index list
    indices_to_keep.extend(filter_close_points(group))

# 4. 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("最終篩選後筆數:", len(final_df))
print(final_df)


原始資料筆數: 6152
包含'交叉口'的資料筆數: 2941
最終篩選後筆數: 2328
                                        id pname   pcode cityname  citycode  \
5     66443903-1565-34ef-a130-9be54de63463   江苏省  320000      无锡市       510   
6     95647b34-0ee0-3dcd-aa3a-0de8e8ab1d16   江苏省  320000      无锡市       510   
8     3e2a2e30-5548-36e0-818e-257ee18281ff   江苏省  320000      无锡市       510   
9     b5d3351e-14e4-368c-a80b-d2e732225d94   江苏省  320000      无锡市       510   
10    734d5425-359b-3704-8680-13c812678209   江苏省  320000      无锡市       510   
...                                    ...   ...     ...      ...       ...   
6040  17f4bcc1-362c-31c8-b815-626223093430   江苏省  320000      无锡市       510   
6061  11bdc83e-9ea5-3a7c-97c6-d12d2b740bf5   江苏省  320000      无锡市       510   
6063  299b49b6-b5eb-3095-85d5-88594ee8d2dc   江苏省  320000      无锡市       510   
6082  f221a583-7f3f-3a21-a64c-328210195d15   江苏省  320000      无锡市       510   
6087  46ce6f10-865a-3322-acca-d5564ea23acd   江苏省  320000      无锡市       510   

     

In [6]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆醫療類資料到 {output_file}")

成功追加 2328 筆醫療類資料到 location_sites.csv


###  交通设施服务

In [7]:
# 1. 讀取 CSV (請確認編碼)
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_交通设施服务.csv' 
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 4688
篩選後筆數: 4422
                                        id pname   pcode cityname  citycode  \
0     0eb6c516-de4b-3f2a-8334-97d0d4233cc0   江苏省  320000      无锡市       510   
1     45040083-35a2-36a9-b745-2358d62136ea   江苏省  320000      无锡市       510   
2     ab50f415-f699-3d2b-9116-d1f11c1b09c1   江苏省  320000      无锡市       510   
3     d753b460-3a98-346e-aea3-9bbc78dad7e6   江苏省  320000      无锡市       510   
4     a5809f64-c349-37e6-aad8-c5befc34ee58   江苏省  320000      无锡市       510   
...                                    ...   ...     ...      ...       ...   
4683  c61f0748-fc59-3031-b261-3b98155e3dff   江苏省  320000      无锡市       510   
4684  9e5bf9b8-e4a9-3153-bb19-bc6eebc485b6   江苏省  320000      无锡市       510   
4685  f72b70d8-c5cb-33ba-baf7-9c1b94f67cc9   江苏省  320000      无锡市       510   
4686  652462f9-7111-3168-b892-5c1d17e80c35   江苏省  320000      无锡市       510   
4687  4e574828-5e5e-3531-9741-0c9cad46d228   江苏省  320000      无锡市       510   

     adname  adcode        n

In [8]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆醫療類資料到 {output_file}")

成功追加 4422 筆醫療類資料到 location_sites.csv


### 医疗保健服务

In [9]:
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_医疗保健服务.csv'
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 2279
篩選後筆數: 2206
                                        id pname   pcode cityname  citycode  \
0     85ce1b43-d360-375c-983a-010634ffb69c   江苏省  320000      无锡市       510   
1     2b9ac8db-b65b-36d5-adc5-c3f9f1b58f29   江苏省  320000      无锡市       510   
2     bc0aa821-9dd4-324d-b0a2-98f2e8d83233   江苏省  320000      无锡市       510   
3     cae65fb6-4baf-3326-8b3e-f8ca59761ae4   江苏省  320000      无锡市       510   
4     091e1bc5-9fa5-3be7-b606-f5221edf20a0   江苏省  320000      无锡市       510   
...                                    ...   ...     ...      ...       ...   
2274  c441b38e-9743-35a2-842c-b465bb2bce79   江苏省  320000      无锡市       510   
2275  a3c0cb23-0635-3edf-89c0-376c3066597d   江苏省  320000      无锡市       510   
2276  31b7d6fd-8556-3fe7-9b45-841758c533fe   江苏省  320000      无锡市       510   
2277  f28c6cb2-d182-3900-8b8d-5d220c128314   江苏省  320000      无锡市       510   
2278  03bf4e53-1808-3118-86b4-18f6459f8df9   江苏省  320000      无锡市       510   

     adname  adcode         

In [10]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆醫療類資料到 {output_file}")

成功追加 2206 筆醫療類資料到 location_sites.csv


### 政府機構

In [11]:
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_政府机构及社会团体.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 3243
篩選後筆數: 2714
                                        id pname   pcode cityname  citycode  \
0     6e151fd2-21d0-31d2-b3ce-fbda303da55c   江苏省  320000      无锡市       510   
1     d6ae2ba1-a392-35e7-8a07-60051e78049c   江苏省  320000      无锡市       510   
2     a8cffc7d-9a17-3aac-9dc1-8a82bedcf4c4   江苏省  320000      无锡市       510   
3     6524a580-1dab-36cc-93fb-c1d157984421   江苏省  320000      无锡市       510   
4     4880de78-3971-3669-af85-3bcd15bc06ac   江苏省  320000      无锡市       510   
...                                    ...   ...     ...      ...       ...   
3238  76135efe-ab4f-3c0d-b425-6b4627c41e3e   江苏省  320000      无锡市       510   
3239  cc3febd6-9ed3-384b-82d2-669b41d7bc8f   江苏省  320000      无锡市       510   
3240  9a6b27da-c90b-3752-b44a-762af84c3195   江苏省  320000      无锡市       510   
3241  b7e79df5-3e6d-30e9-95ab-43035afd07d8   江苏省  320000      无锡市       510   
3242  771e1d8f-6956-3cc5-ba8c-92b71a17279d   江苏省  320000      无锡市       510   

     adname  adcode         

In [12]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆政府机构資料到 {output_file}")

成功追加 2714 筆政府机构資料到 location_sites.csv


### 公共設施

In [13]:
file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_公共设施.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 566
篩選後筆數: 552
                                       id pname   pcode cityname  citycode  \
0    35fd8a75-f4e7-368b-a475-8c93282e501c   江苏省  320000      无锡市       510   
1    7d875ced-e88b-31c4-a9ca-633899e94bd0   江苏省  320000      无锡市       510   
2    72594756-f6b7-3118-a5c9-b03f703bb319   江苏省  320000      无锡市       510   
3    51410705-f08b-39e8-82f6-7db32c1144bb   江苏省  320000      无锡市       510   
4    9b268842-f9ae-3977-9ccf-2b9a5eee197e   江苏省  320000      无锡市       510   
..                                    ...   ...     ...      ...       ...   
561  84a8acd6-7b00-39a6-a709-9d1b1a6899cc   江苏省  320000      无锡市       510   
562  0ab4d759-f61c-36f6-9e6a-c1d5962f9417   江苏省  320000      无锡市       510   
563  21316736-ec42-3b0f-a841-d1471bd6c9d8   江苏省  320000      无锡市       510   
564  b15b72ef-9586-3049-b58b-d1ad24ce0ca1   江苏省  320000      无锡市       510   
565  44830a8f-8439-3cfd-9aec-1bcb0c46da99   江苏省  320000      无锡市       510   

    adname  adcode        name            

In [14]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆政府机构資料到 {output_file}")

成功追加 552 筆政府机构資料到 location_sites.csv


### 廣場或公園

In [15]:
import pandas as pd

file_path = 'Yixing_data/2024_江苏省_无锡市_宜兴市_风景名胜.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')

# ==========================================
# 新增步驟：先篩選 name 中包含特定關鍵字的資料
# ==========================================

# 定義關鍵字 (使用 | 符號代表 "或")
keywords = '公园|广场' 

# 建立遮罩 (Mask)：找出 name 欄位中包含這些字的列
# na=False 確保如果 name 是空的 (NaN) 不會報錯，直接視為 False
mask = df['name'].str.contains(keywords, na=False)

# 產生只包含公園或廣場的臨時 DataFrame
target_df = df[mask]


indices_to_keep = []

# 注意：這裡改成對 target_df (篩選後的資料) 進行 groupby
for name, group in target_df.groupby('address'):
    # 假設 filter_close_points 函式已經在上面定義好了
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
# loc 會根據保留下來的 index 回到原始 df (或 target_df) 抓取資料
final_df = df.loc[indices_to_keep].sort_index()

print("原始總筆數:", len(df))
print(f"符合關鍵字 '{keywords}' 的筆數:", len(target_df))
print("經距離篩選後的最終筆數:", len(final_df))
print(final_df)

原始總筆數: 679
符合關鍵字 '公园|广场' 的筆數: 107
經距離篩選後的最終筆數: 107
                                       id pname   pcode cityname  citycode  \
0    31ca1582-f0ec-3d40-b77a-dcac77fc0601   江苏省  320000      无锡市       510   
11   715b0d83-c323-3575-8f3f-d0bb4dbbcd8b   江苏省  320000      无锡市       510   
21   626f667f-b235-3b6c-be22-d5b13247afd0   江苏省  320000      无锡市       510   
26   7070b370-50c0-3fdf-ba3e-ce8575e99807   江苏省  320000      无锡市       510   
27   32c09e56-1ebe-366e-80c8-96171aabf74d   江苏省  320000      无锡市       510   
..                                    ...   ...     ...      ...       ...   
636  d459d453-bc31-3882-97be-b5f503f70563   江苏省  320000      无锡市       510   
638  8c5c1adf-87c3-35f8-b5ae-0a74266c6c43   江苏省  320000      无锡市       510   
641  fabf1e4e-bce5-39c7-b198-574bd9024ee7   江苏省  320000      无锡市       510   
650  d9b3aaa4-1b1c-36e2-bbf8-c148f2272046   江苏省  320000      无锡市       510   
652  bf738b43-94b0-3c10-80f4-3912d629d10f   江苏省  320000      无锡市       510   

    adname  

In [16]:
import os
output_file = 'location_sites.csv'

# 檢查檔案是否存在
file_exists = os.path.exists(output_file)

final_df.to_csv(
    output_file, 
    mode='a',                # 追加模式
    index=False, 
    header=not file_exists,  # 若檔案存在則不寫標題
    encoding='utf-8-sig'
)

print(f"成功追加 {len(final_df)} 筆政府机构資料到 {output_file}")

成功追加 107 筆政府机构資料到 location_sites.csv


### 總整理一次

In [17]:
file_path = 'location_sites.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='gb18030')


# 1. 對 address 進行 groupby
# 2. 對每個 group 應用篩選邏輯
# 3. 收集所有要保留的 index
indices_to_keep = []
for name, group in df.groupby('address'):
    indices_to_keep.extend(filter_close_points(group))

# 取得最終結果
final_df = df.loc[indices_to_keep].sort_index()

print("原始筆數:", len(df))
print("篩選後筆數:", len(final_df))
print(final_df)

原始筆數: 13628
篩選後筆數: 13536
                                         id pname   pcode cityname  citycode  \
0      3afcb146-d32d-3622-8838-5e6025dfcffa   江苏省  320000      无锡市       510   
1      ef1ac6e5-99f4-3cab-8b71-1860d3b4825b   江苏省  320000      无锡市       510   
2      59474927-1d35-3859-bd15-ec527d7f06a3   江苏省  320000      无锡市       510   
3      6e2c351f-8dfe-302b-8b68-e5932c16ee8a   江苏省  320000      无锡市       510   
4      c295a58b-d914-3463-8c76-207003da92de   江苏省  320000      无锡市       510   
...                                     ...   ...     ...      ...       ...   
13623  d459d453-bc31-3882-97be-b5f503f70563   江苏省  320000      无锡市       510   
13624  8c5c1adf-87c3-35f8-b5ae-0a74266c6c43   江苏省  320000      无锡市       510   
13625  fabf1e4e-bce5-39c7-b198-574bd9024ee7   江苏省  320000      无锡市       510   
13626  d9b3aaa4-1b1c-36e2-bbf8-c148f2272046   江苏省  320000      无锡市       510   
13627  bf738b43-94b0-3c10-80f4-3912d629d10f   江苏省  320000      无锡市       510   

      adname  